# GPU Programming Lab 2
This lab is for using GPU programming with cuda using python to perform machine learning task. The consists of 1 exercise and a homework. 

- <font color='red'><b> After each exercise, write a detailed summary explaining what you have done, your observations and  conclusions. </b></font>
- <font color='red'><b> Make sure to write your name and your partner name (as registred in Halmstad University) in the name section above. </b></font>
    
- <font color='red'><b> You can do the lab in a group of a maximum of two students. </b></font>

- <font color='red'><b> Only one of the students upload the lab to the blackboard. </b></font>

# CUDA
CUDA is a parallel programming platform and an API that facilitates the access to the CUDA-Enabled GPU functuonality for general purpose computing. It allows speeding up the software by utilizing the GPU power for the parallelizable part of the computation. Many Deep Learning platforms like tenserflow, keras, pytorch and others, rely on CUDA for their computations.

## Common CUDA terminology:
- <b>Host:</b> The CPU
- <b>Device:</b> The GPU
- <b>Host Memory:</b> The system main memory
- <b>Device Memory:</b> The GPU onboard memory
- <b>kernel:</b> A function that runs on the Device

Threads are organized into a grid of blocks, where each block contains a subset of the threads that can cooperate using a block shared memory and can synchronize within each block.

<img src='https://drive.google.com/uc?id=1QzXBVWki0M80KKY_CPzQu1ivE3fAcf2U' width="50%" height="50%"></img>


Parallel portions of an application are executed on the device (GPU) as kernels, where an array of threads excutes each kernel. Each thread has an ID, by which it controls the portion of the data to excute the Kernel. All threads runs the same code on different portions of the data. Grids and Blocls can be organized as 1D, 2D, or 3D arrays. 

<img src='https://drive.google.com/uc?id=1vqh749XFQhfZwq7m7E-VXscBblh58mei' width="50%" height="50%"></img>






# Numba
CUDA is designed to work with C++, but in this Lab we will work with Numba; a Python JIT compiler that translates subsets of the code into machine code, and enables writing a parallel GPU algorithms in Python

## Numba installation


conda install numba

pip install numba

# Kernel 
- A Kernel is declared as a function with @cuda.jit decorator.
- A Kernel function cannot have a return value and manages outputs as input-output arrays

In [1]:
import numpy as np
import math
import numba as nb
from numba import cuda
import time

In [2]:
# kernel decleration
@cuda.jit
def my_kernel(io_array):
    # code here
    pass


To invoc a kernal you have to specify number of blocks in the grid, and the number of threads per block. This can be done by specifying the number of threads per block and calculating how many blocks are required in the grid based on the size of the data.

<font color=red>Important note: In the case that the data size is not divisable by the the number of thread per block, we take the ceiling of the number to reserve an extra block for the remaining part of the data. So the threads in the last block will not be fully occupied.</font>

In [3]:
# kernel invocation
data = np.ones(256)

threadsperblock = 32
blockspergrid = math.ceil(len(data)/threadsperblock)

my_kernel[blockspergrid, threadsperblock](data)


/usr/local/lib/python3.10/dist-packages/numba/cuda/dispatcher.py:488: NumbaPerformanceWarning: Grid size 8 will likely result in GPU under-utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))
/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


## Exercise *1*: Distance Matrix
The distance matrix (D) of a data matrix (A) is the matrix that contains the eucleadian distance between each two row vectors as shown in the following figure.
<img src='https://drive.google.com/uc?id=1UMMRYmtPW9_Tonq20GBjxsDLrNFYSTdc' width="50%" height="50%"></img>

where 
$$D[i,j]=D[j,i]=dist(A[i,:], A[j,:])$$


Use what you have learned in the previous exercises to write a kernel and a host function to compute the distance matrix of a data matrix. 


In [4]:
import numpy as np
import math
from numba import cuda

# Kernel to calculate Euclidean distance
@cuda.jit(device=True)
def euclidean_dist(x, y, n):
    sum_squares = 0.0
    for i in range(n):
        diff = x[i] - y[i]
        sum_squares += diff * diff
    return math.sqrt(sum_squares)

# Kernel to calculate distance matrix
@cuda.jit
def distance_matrix(mat, out):
    m = mat.shape[0]  # Number of rows
    n = mat.shape[1]  # Number of columns

    i, j = cuda.grid(2)
    if i < m and j < m:
        out[i, j] = euclidean_dist(mat[i], mat[j], n)

def gpu_dist_matrix(mat):
    m = mat.shape[0]  # Number of rows

    # Allocate output array on the device
    out = cuda.device_array((m, m))

    # Define blocks and grid dimensions
    threads_per_block = (16, 16)
    blocks_per_grid_x = math.ceil(mat.shape[0] / threads_per_block[0])
    blocks_per_grid_y = math.ceil(mat.shape[0] / threads_per_block[1])
    blocks_per_grid = (blocks_per_grid_x, blocks_per_grid_y)

    # Run the kernel
    distance_matrix[blocks_per_grid, threads_per_block](mat, out)

    return out.copy_to_host()  # Transfer the result back to the host

A = np.random.randn(1024,1024)
D = gpu_dist_matrix(A)
print(D)


[[ 0.         43.99124551 42.35231415 ... 44.36805577 45.27301867
  44.10859555]
 [43.99124551  0.         43.6285848  ... 45.17765575 46.65116444
  44.58858591]
 [42.35231415 43.6285848   0.         ... 44.03962269 45.88730286
  43.64291063]
 ...
 [44.36805577 45.17765575 44.03962269 ...  0.         46.92075631
  44.81056607]
 [45.27301867 46.65116444 45.88730286 ... 46.92075631  0.
  44.01762329]
 [44.10859555 44.58858591 43.64291063 ... 44.81056607 44.01762329
   0.        ]]


/usr/local/lib/python3.10/dist-packages/numba/cuda/cudadrv/devicearray.py:885: NumbaPerformanceWarning: Host array used in CUDA kernel will incur copy overhead to/from device.
  warn(NumbaPerformanceWarning(msg))


# Homework: K-Nearest Neighbors (GPU version)

K-Nearest Neighbors is one of the simplest and most intuitive algorithms in machine learning that relies on the principle that close points behave similarly. It is one of the case-based learning algorithms that can learn non-linear complicated decision boundaries with a single hyperparameter, i.e. K the number of nearest neighbors. The problem of this algorithm is that, to find the k nearest neighbors of a specific point, you have to compute the distances to all the points in the training dataset, which is very costly in terms of computation especially with a large amount of data. A great benefit can be achieved by performing such computation on the GPU.

Your task is to implement the K-Nearest Neighbors algorithm using python, and Numba, and CUDA programming.
Identify the parts of the algorithm that can make use of the GPU and implement them as CUDA kernels.

Use the MNIST dataset as an example and implement a K-Nearest Neighbors classifier to classify the image of the digit into its category.

Try different numbers of K and figure out the number that maximizes the accuracy of the classifier.
Build another K-Nearest Neighbors using the Sciket-learn library and compare the computation time with your GPU-enabled algorithm. 

You can download MNIST from Keras library: ( https://keras.io/api/datasets/mnist/ )


In [59]:
#Importants imports
import numpy as np
import math
import tensorflow as tf
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.datasets import mnist
from numba import cuda
import time

In [55]:
# Define a CUDA kernel using the @cuda.jit() decorator
@cuda.jit()
def dist_matrix(x, y, z):
    # Obtain the current thread indices
    row, col = cuda.grid(2)

    # Check if the indices are within the bounds of the output matrix
    if z.shape[1] > col and z.shape[0] > row:
        sum = 0
        # Compute the squared Euclidean distance between the corresponding points
        for n in range(y.shape[1]):
            diff = y[row, n] - x[col, n]
            sum += diff * diff
        # Store the result in the output matrix
        z[row, col] = sum


def GPU_Euclidean_distance(x, y):
    # Transfer input arrays to the GPU memory
    A_ = cuda.to_device(x)
    B_ = cuda.to_device(y)

    # Define the thread block size
    thread_block_size = (32, 32)

    # Compute the grid size based on the input array shapes
    blockX = math.ceil(B_.shape[0] / thread_block_size[0])
    blockY = math.ceil(A_.shape[0] / thread_block_size[1])
    blockGrid = (blockX, blockY)

    # Allocate GPU memory for the output matrix
    C_ = cuda.device_array((y.shape[0], x.shape[0]), np.float32)

    # Launch the CUDA kernel to compute the distance matrix
    dist_matrix[blockGrid, thread_block_size](A_, B_, C_)

    # Copy the result back to the CPU memory
    Cop = C_.copy_to_host()

    # Return the computed distance matrix
    return Cop


In [56]:
# Load the MNIST dataset 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")

# Perform shape assertions to ensure the expected shapes of the loaded data
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

# Normalize the pixel values to the range of [0, 1]
x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

# Reshape the data to flatten the images into a 1D array
x_train = x_train.reshape((60000, 28*28))
x_test = x_test.reshape((10000, 28*28))


In [57]:
def pred(distance, X):
    # Get the number of instances in the distance matrix
    numbers = distance.shape[0]
    
    # Create an array to store the predicted labels
    predicting = np.zeros(numbers)
    
    # Iterate over each instance
    for m in range(numbers):
        # Find the nearest neighbors based on the sorted distances
        neighbors = np.take(y_train, np.argsort(distance[m]))[:X]
        
        # Count the occurrences of each label in the neighbors
        (labels, counts) = np.unique(neighbors, return_counts=True)
        
        # Predict the label based on the majority vote
        predicting[m] = labels[np.argmax(counts)]
    
    return predicting


def acc(predicting, y_test):
    # Initialize a counter for correct predictions
    counter = 0
    
    # Iterate over each instance in the test set
    for m in range(len(y_test)):
        # Check if the prediction matches the true label
        if predicting[m] == y_test[m]:
            counter += 1
    
    # Calculate and print the accuracy
    accuracy = (counter / len(predicting)) * 100
    return print("Accuracy =", accuracy, "%")


In [60]:
start_time = time.time()
dest = GPU_Euclidean_distance(x_train,x_test)
print(dest)
k = 3
predd = pred(dest, k)
acc(predd, y_test)
end_time = time.time()
gpu_time = end_time - start_time
print("GPU time: {} seconds".format(gpu_time))


[[ 88.27123  108.04831   89.12162  ...  89.63276   90.89498   85.06571 ]
 [125.059364 131.12627  135.88385  ... 126.7901   109.497025 112.222626]
 [ 85.077866 101.53147   83.78076  ...  85.681335  71.28048   70.412415]
 ...
 [106.175934 131.89719  110.35838  ...  99.73732  106.32168   94.05682 ]
 [111.56955  108.89341  121.26767  ... 108.294334  82.567276  85.4585  ]
 [133.30196   97.091736 157.6864   ... 146.9399    65.447334 135.59558 ]]
Accuracy = 97.05 %
GPU time: 66.87765145301819 seconds


In [61]:
'''K-Nearest Neighbors using the Sciket-learn library '''

# load the mnist dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz")
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

# reshape the images into one-dimensional arrays
x_train = x_train.reshape(-1, 784)
X_test = X_test.reshape(-1, 784)

# perform feature scaling
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
X_test = scaler.transform(X_test)

# train the KNN model
knn = KNeighborsClassifier(n_neighbors=3)
start_time = time.time()
knn.fit(x_train, y_train)

# make predictions
y_pred = knn.predict(X_test)

# evaluate the model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

end_time = time.time()
cpu_time = end_time - start_time
print("CPU time: {} seconds".format(cpu_time))


              precision    recall  f1-score   support

           0       0.33      0.00      0.00       980
           1       0.11      1.00      0.20      1135
           2       1.00      0.00      0.00      1032
           3       0.00      0.00      0.00      1010
           4       0.00      0.00      0.00       982
           5       0.00      0.00      0.00       892
           6       0.40      0.00      0.00       958
           7       1.00      0.00      0.00      1028
           8       0.00      0.00      0.00       974
           9       0.00      0.00      0.00      1009

    accuracy                           0.11     10000
   macro avg       0.28      0.10      0.02     10000
weighted avg       0.29      0.11      0.02     10000

[[   1  979    0    0    0    0    0    0    0    0]
 [   0 1135    0    0    0    0    0    0    0    0]
 [   0 1026    2    1    0    0    3    0    0    0]
 [   0 1010    0    0    0    0    0    0    0    0]
 [   1  981    0    0    0   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
